# Будем искать способы отфильровать нерелевантные вакансии

## Разметка

По поводу толоки. Получилось не дешево. За одну задачу (для 3-ех раз размеченную) 0.019$ - то есть для 12_000 записей это 228$, даже если взять треть, то это дорого для этой задачи

В общем проще в Эксель (а точнее плагин для редактирования CSV таблицей)

In [60]:
import pandas as pd
import pickle
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from pymystem3 import Mystem
import nltk
from nltk.corpus import stopwords
import lightgbm as lgb
from typing import List, Set

In [2]:
df = pd.read_csv('../data/processed/vacancies.csv', encoding='utf-8')[['vacancy_id', 'name', 'query']]
df.head()

,vacancy_id,name,query
0,77630537,Сборщик электрических машин и аппаратов / ученик,['машинное обучение']
1,80150068,Швея,['машинное обучение']
2,80485628,Аналитик,"['аналитик данных', 'CV']"
3,80628991,Менеджер объекта клининга,['машинное обучение']
4,80192310,"Официант в ресторан ""She"" (на Большой Никитской)",['Artificial intelligence']


In [40]:
with open('../data/features/prof_index_to_prof_name.pkl', 'rb') as f:
    prof_index_to_prof_name = pickle.load(f)

with open('../data/features/quety_to_prof_index.pkl', 'rb') as f:
    quety_to_prof_index = pickle.load(f)

def add_prof_set_to_df(df):
    r = re.compile('\\\\|////|,')
    def query_to_prof_set(s):
        query_set = {x.strip(" '") for x in re.split(r, s.strip('[]'))} - {''}
        prof_set = set([])
        for q in query_set:
            i = quety_to_prof_index[q]
            p = prof_index_to_prof_name[i]
            prof_set.add(p)
        return prof_set

    df['prof_set'] = df['query'].apply(query_to_prof_set)

    return df

In [11]:
df['actual'] = 0
df = add_prof_set_to_df(df)

df.head()



,vacancy_id,name,query,actual,prof_set
0,77630537,Сборщик электрических машин и аппаратов / ученик,['машинное обучение'],0,{ML инженер}
1,80150068,Швея,['машинное обучение'],0,{ML инженер}
2,80485628,Аналитик,"['аналитик данных', 'CV']",0,"{Аналитик, ML инженер}"
3,80628991,Менеджер объекта клининга,['машинное обучение'],0,{ML инженер}
4,80192310,"Официант в ресторан ""She"" (на Большой Никитской)",['Artificial intelligence'],0,{Data scientist}


**Файл filtring_ds.csv используется для ручной разметки актуальных вакансий**

In [12]:
df.to_csv('data/filtring_ds.csv', index=False, encoding='utf-8')

разметка в файле `filtring_ds` производилась вручную

In [3]:
fintring_df = pd.read_csv('data/filtring_ds.csv', encoding='utf-8')
fintring_df.head()

,vacancy_id,name,query,actual,prof_set
0,77630537,Сборщик электрических машин и аппаратов / ученик,['машинное обучение'],0,{'ML инженер'}
1,80150068,Швея,['машинное обучение'],0,{'ML инженер'}
2,80485628,Аналитик,"['аналитик данных', 'CV']",1,"{'Аналитик', 'ML инженер'}"
3,80628991,Менеджер объекта клининга,['машинное обучение'],0,{'ML инженер'}
4,80192310,"Официант в ресторан ""She"" (на Большой Никитской)",['Artificial intelligence'],0,{'Data scientist'}


In [4]:
# постразметка

pd.set_option('display.max_rows', None)

def simplify(s):
    return s.lower(). \
                replace('"', ' '). \
                replace(",", ' '). \
                replace('(', ' '). \
                replace(')', ' '). \
                replace('/', ' '). \
                replace('.', ' ')

ww = ['аналитик', 'analy', 'dwh', 'MLOps', ' ml ', ' ai ', ' cv ', ' nlp ',
      'Data Engineer', 'Data Office', 'Data Science', 'DataOps', 'Data Quality',
      'Big Data', 'Data Lake', 'DataEngineer', 'баз данных', 'ETL']
d = fintring_df[fintring_df.name.apply(
    lambda x: any([(w.lower() in simplify(x)) for w in ww])) & (fintring_df.actual == 0)]
#fintring_df.loc[d.index, 'actual'] = 1

print('Rows count:', d.shape[0])
list(d['name'])


Rows count: 0


[]

In [112]:
fintring_df.to_csv('data/filtring_ds.csv', index=False, encoding='utf-8')

## Разделение выборок, базовая модель, метрика

In [5]:
fintring_df.actual.value_counts()

0    9148
1    3366
Name: actual, dtype: int64

In [6]:
RANDOM_SEED = 42

data_f = fintring_df[['name']]
data_t = fintring_df['actual']

train_f, test_f, train_t, test_t = train_test_split(data_f, data_t, 
                                                  train_size=0.75, random_state=RANDOM_SEED, stratify=data_t)

print('train_f shape = ', train_f.shape)
print('train_t shape = ', train_t.shape)
print('test_f shape = ', test_f.shape)
print('test_t shape = ', test_t.shape)

train_f shape =  (9385, 1)
train_t shape =  (9385,)
test_f shape =  (3129, 1)
test_t shape =  (3129,)


In [123]:
# dummy classifier
dummy = DummyClassifier()
dummy.fit(train_f, train_t)
predict = dummy.predict(test_f)
f1_score(test_t, predict)

0.0

In [138]:
# keyword baseline model

key_words = ['аналитик', 'analy', 'dwh', 'MLOps', ' ml ', ' ai ', ' cv ', ' nlp ',
      'Data Engineer', 'Data Office', 'Data Science', 'DataOps', 'Data Quality',
      'Big Data', 'Data Lake', 'DataEngineer', 'баз данных', 'ETL',
      'инженер данных', 'computer vision', 'database']

def keyword_predict(X):
      def simplify(s):
            return s.lower(). \
                replace('"', ' '). \
                replace(",", ' '). \
                replace('(', ' '). \
                replace(')', ' '). \
                replace('/', ' '). \
                replace('.', ' ')
      
      def predict_row(r):
            return any([(w.lower() in simplify(r)) for w in key_words])

      y = X.name.apply(lambda x: predict_row(x))
      return y

predict = keyword_predict(test_f)
score = f1_score(test_t, predict)
print('Baseline f1 =', score)

    

Baseline f1 = 0.9368686868686869


In [29]:
# Words bag Vectorizers

class RelevantVacancyClassifier():
    """Класссификатор вакансий, которые относятся к DS
    на вход dataframe c одним столбцом 'name'
    на выходе актуальность (0/1)
    """

    def __init__(self):

        nltk.download('stopwords')
        self._stop_words = set(stopwords.words('english')).union(stopwords.words('russian'))

        self._m = Mystem()

    def _lemmatize(self, X):
        def simplify(s):
            return s.lower(). \
                replace('"', ' '). \
                replace(",", ' '). \
                replace('(', ' '). \
                replace(')', ' '). \
                replace('\\', ' '). \
                replace('-', ' '). \
                replace('/', ' '). \
                replace('.', ' ')
        
        X = [[x.strip() for x in self._m.lemmatize(simplify(r)) if x.strip() != ''] for r in X.name]
        return [' '.join(x) for x in X]

    def fit(self, X, y):

        X = self._lemmatize(X)

        self._vectorizer = CountVectorizer(stop_words=list(self._stop_words))
        #self._vectorizer = TfidfVectorizer(stop_words=list(self._stop_words))

        #self._model = LogisticRegression(class_weight='balanced')
        self._model = lgb.LGBMClassifier(n_estimators=500, random_state=RANDOM_SEED)

        X = self._vectorizer.fit_transform(X)
        self._model.fit(X.astype(float), y)

    def predict(self, X):
        
        X_lemm = self._lemmatize(X)
        XX = self._vectorizer.transform(X_lemm)
        y = self._model.predict(XX.astype(float))

        # коррекция на базе правил
        key_words = ['NLP', 'MLOps', 'DataOps', 'Computer Vision']
        for i, x in enumerate(X_lemm):
            if any([(w.lower() in x.split(' ')) for w in key_words]):
                y[i] = 1

        return y
    
    def dump(self, filename: str):
        dumped = [self._vectorizer, self._model]
        with open(filename, 'wb') as f:
            pickle.dump(dumped, f)

    def load(self, filename: str):
        with open(filename, 'rb') as f:
            dumped = pickle.load(f)
            self._vectorizer = dumped[0]
            self._model = dumped[1]

clf = RelevantVacancyClassifier()
clf.fit(train_f, train_t)
predict = clf.predict(test_f)
score = f1_score(test_t, predict)
print('Vectorizer f1 =', score)

[nltk_data] Downloading package stopwords to /Users/dima/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Vectorizer f1 = 0.9699879951980791


In [8]:
# посмотрим где предикт модели отличается от разметки

df = fintring_df.copy()
df['predict'] = clf.predict(df[['name']])


In [9]:
d = df[df.predict != df.actual]
print(d.shape[0])
d[['name', 'query', 'actual', 'predict']].head() # просматривались все данные

102


,name,query,actual,predict
97,Technical Office Engineer / Senior Cost Contro...,['Advanced Analytics'],0,1
115,Market Intelligence manager (Senior),"['data analyst', 'business analyst', 'data', '...",1,0
195,Менеджер по продажам решений RPA/BI,['business analyst'],0,1
222,Senior Python developer,['GPT'],1,0
250,Технический писатель (hadoop),['аналитик данных'],1,0


In [30]:
# посмотрев предикты видим, что предсказания даже более актуальные чем разметка, кроме редких кейсов в меньшенстве
# поэтому обучем эту модель на всех данных (можно сериализовать для прода) и обновим предсказания

# посмотрим где предикт модели отличается от разметки

df = fintring_df.copy()
clf = RelevantVacancyClassifier()
clf.fit(df[['name']], df['actual'])
df['predict'] = clf.predict(df[['name']])


[nltk_data] Downloading package stopwords to /Users/dima/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
d = df[df.predict != df.actual]
print(d.shape[0])
d[['name', 'query', 'actual', 'predict']].head() 

58


,name,query,actual,predict
115,Market Intelligence manager (Senior),"['data analyst', 'business analyst', 'data', '...",1,0
222,Senior Python developer,['GPT'],1,0
250,Технический писатель (hadoop),['аналитик данных'],1,0
271,Бизнес-ассистент / помощник руководителя,['аналитик данных'],1,0
296,Network Administrator (rotation),['data engineer'],0,1


In [32]:
# сериализуем
clf.dump('../models/RelevantVacancyClassifier.pkl')

In [34]:
# проверяем
lc = RelevantVacancyClassifier()
lc.load('../models/RelevantVacancyClassifier.pkl')

df = fintring_df.copy()
df['predict'] = lc.predict(df[['name']])

d = df[df.predict != df.actual]
print(d.shape[0])
d[['name', 'query', 'actual', 'predict']].head() 

[nltk_data] Downloading package stopwords to /Users/dima/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


58


,name,query,actual,predict
115,Market Intelligence manager (Senior),"['data analyst', 'business analyst', 'data', '...",1,0
222,Senior Python developer,['GPT'],1,0
250,Технический писатель (hadoop),['аналитик данных'],1,0
271,Бизнес-ассистент / помощник руководителя,['аналитик данных'],1,0
296,Network Administrator (rotation),['data engineer'],0,1


**!!! Модель конечно нужно переобучить на всем корпусе, включить в пайпллан прода, серилизовать и т.д.**

## Проверяем/дополняем профессии

In [150]:
df = pd.read_csv('../data/processed/vacancies.csv', encoding='utf-8')
df = add_prof_set_to_df(df)
df['prof_set_rulebase'] = [set([]) for _ in range(len(df))]
df = df[['vacancy_id', 'name', 'prof_set', 'prof_set_rulebase', 'query']]

df.head()

,vacancy_id,name,prof_set,prof_set_rulebase,query
0,80485628,Аналитик,"{ML инженер, Аналитик}",{},"['аналитик данных', 'CV']"
1,80652370,Игровой аналитик (продуктовый аналитик),"{Аналитик, Data scientist, Big Data}",{},"['аналитик данных', 'data analyst', 'data', 'B..."
2,78278410,Главный системный аналитик/Senior system Analy...,{Аналитик},{},['data analyst']
3,80585316,Бизнес-аналитик (транзакционные продукты),{Аналитик},{},['business analyst']
4,80223554,Маркетолог-аналитик,"{Инженер данных, Аналитик, Data scientist}",{},"['data analyst', 'data mining', 'data']"


In [158]:
# rule based

xdf = df.copy()



def _simplify(s):
        return s.lower(). \
            replace('"', ' '). \
            replace(",", ' '). \
            replace('(', ' '). \
            replace(')', ' '). \
            replace('\\', ' '). \
            replace('-', ' '). \
            replace('/', ' '). \
            replace('.', ' ')



#xdf['lemm_name'] = _lemmatize(xdf)

def apply_rule_and(df: pd.DataFrame, prof: str, keywords_list: List[str] = None, only_empty: bool = False) -> pd.DataFrame:
    if keywords_list is None:
        keywords_list = [prof]

    for keywords in keywords_list:
        keywords = re.split(' |-', keywords)

        rule = df.apply(lambda x: 
                        all([(kw.lower() in x['name'].lower()) for kw in keywords]) and
                        (only_empty == False or len(x['prof_set_rulebase']) == 0)
                        , axis=1)
        filter = df[rule].index

        df.loc[filter, 'prof_set_rulebase'] = \
            df.loc[filter, 'prof_set_rulebase'].apply(lambda x: x.union(set([prof])))

    return df

def apply_rule_kw(df: pd.DataFrame, prof: str, keywords: List[str], only_empty: bool = False) -> pd.DataFrame:
    for kw in keywords:
        kw = f' {kw.lower()} '
        rule = df.apply(lambda x:
                        kw in ' '+_simplify(x['name'])+' ' and
                        (only_empty == False or len(x['prof_set_rulebase']) == 0)
                        , axis=1)
        filter = df[rule].index

        df.loc[filter, 'prof_set_rulebase'] = \
                df.loc[filter, 'prof_set_rulebase'].apply(lambda x: x.union(set([prof])))

    return df

# Аналитики
xdf = apply_rule_and(xdf, 'Системный аналитик')
xdf = apply_rule_and(xdf, 'Бизнес-аналитик', ['Бизнес-аналитик', 'Business Analyst'])
xdf = apply_rule_and(xdf, 'Аналитик данных', ['Аналитик данных', 'Data Analyst'])
xdf = apply_rule_and(xdf, 'Продуктовый аналитик')
xdf = apply_rule_and(xdf, 'Аналитик', ['Аналитик', 'Analyst'], only_empty=True)

# Администратор баз данных
xdf = apply_rule_and(xdf, 'Администратор баз данных', ['Администратор баз данных', 'Администратор БД'])

# Инженеры данных
xdf = apply_rule_and(xdf, 'Инженер данных', ['Инженер данных', 'Data Engineer', 
                'Дата инженер', 'Data инженер',
                'Data Architect', 'Hadoop', 'Kafka'
])
xdf = apply_rule_kw(xdf, 'Инженер данных', ['баз данных', 'PostgreSQL', 'MSSQL'], only_empty=True)
xdf = apply_rule_kw(xdf, 'Инженер данных', ['Hadoop', 'Kafka'])


# Дата Сайенс
xdf = apply_rule_and(xdf, 'Data Scientist')

# ML инженер
xdf = apply_rule_kw(xdf, 'ML инженер', ['ML', 'ETL', 'MLOps'])

# Big Data
xdf = apply_rule_kw(xdf, 'Big Data', ['Big Data', 'Биг Дата', 'DWH', 'Data lake', 'Hadoop', 'Kafka',
                                      'больших данных', 'большие данные'])

# остатки
xdf = apply_rule_and(xdf, 'Data Scientist', ['Data Science'])
xdf = apply_rule_and(xdf, 'Computer Vision', ['Computer Vision', 'CV'])
xdf = apply_rule_and(xdf, 'NLP', ['Computer Vision', 'NLP', 'Natural Language Processing'])

processed = xdf[xdf.prof_set_rulebase.apply(lambda x: x != set([]))].shape[0]

rule = xdf.prof_set_rulebase.apply(lambda x: x == set([]))
filter = xdf[rule].index
xdf.loc[filter, 'prof_set_rulebase'] = xdf.loc[filter, 'prof_set']

xdf.to_csv('data/prof_ds.csv', index=False, encoding='utf-8')

# Total: 3339, processed: 3133
print(f'Total: {xdf.shape[0]}, processed: {processed}')


Total: 3339, processed: 3133


- проверить ещй раз
- посмотреть на count vectorizer c фразами 
- посмотреть а статистику профессий
- в пайплайн

In [142]:
xdf[xdf.prof_set_rulebase.apply(lambda x: len(x) == 0)].name.value_counts()[:20]

Специалист Центра подготовки данных                                                                 4
Senior Devops engineer                                                                              3
BI разработчик                                                                                      3
Специалист по разметке данных со знанием разработки                                                 3
Python Developer (Data Office)                                                                      2
Специалист по сверке данных                                                                         2
Специалист по аналитической поддержке                                                               2
Разработчик BI                                                                                      2
Руководитель направления BI                                                                         2
Senior DevOps Engineer                                                            